<a href="https://colab.research.google.com/github/toussaintma/neuralnetworksfromzerotohero/blob/main/walkthrough_makemore_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# data and code at https://github.com/karpathy/makemore
# NanoGPT code at https://github.com/karpathy/nanoGPT
# course at https://www.youtube.com/watch?v=PaCmpygFfXo&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=2&t=11s
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# Letw built GTP: from scratch, in code, spelled out
# GPT 3 paper: https://arxiv.org/pdf/2005.14165.pdf
# Attention is all you need paper: https://arxiv.org/abs/1706.03762


--2023-07-19 15:44:22--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-07-19 15:44:22 (20.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [8]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

print('Length of dataset in characters: ', len(text))

Length of dataset in characters:  1115394


In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab = ''.join(chars)
print(f'Vocabulary of size {vocab_size}: {vocab}')

Vocabulary of size 65: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [36]:
stoi = {s:i for (i, s) in enumerate(chars)}
itos = {i:s for (i, s) in enumerate(chars)}
encode = lambda x: [stoi[a] for a in x]
decode = lambda x: ''.join([itos[a] for a in x])
stoi[itos[13]], decode(encode('ABC'))

(13, 'ABC')

In [38]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [43]:
n = int(0.90 * len(data))
train_data = data[:n]
eval_data = data[n:]

In [44]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [45]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f'when input i {context} the target is {target}')

when input i tensor([18]) the target is 47
when input i tensor([18, 47]) the target is 56
when input i tensor([18, 47, 56]) the target is 57
when input i tensor([18, 47, 56, 57]) the target is 58
when input i tensor([18, 47, 56, 57, 58]) the target is 1
when input i tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input i tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input i tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [54]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_data(split):
  data = train_data if split == 'train' else eval_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i: i+block_size] for i in ix], dim=0)
  y = torch.stack([data[i+1: i+block_size+1] for i in ix], dim=0)
  return x,y

get_data('train')

(tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))